# Install requirement for crawl4ai

In [ ]:
!crawl4ai-setup
!crawl4ai-doctor
!playwright install

# Load Requirements

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from crawl4ai import AsyncWebCrawler, LXMLWebScrapingStrategy
from crawl4ai.async_configs import BrowserConfig, CrawlerRunConfig

# Scraping

In [ ]:
url = 'https://www.ipthailand.go.th/th/trademark-002.html'

In [ ]:
async def crawler_function(url):
    browser_config = BrowserConfig(verbose=False) # verbose for logging

    run_config = CrawlerRunConfig(
      # word_count_threshold=10,        # Minimum words per content block
      exclude_external_links=True,    # Remove external links
      remove_overlay_elements=True,   # Remove popups/modals
      process_iframes=True            # Process iframe content
      )

    async with AsyncWebCrawler(config=browser_config) as crawler:
        result = await crawler.arun(
            url=url,
            config=run_config
        )
        # print(result.markdown)  # Print clean markdown content
        return result

In [ ]:
result = await crawler_function(url)

In [ ]:
print(result.markdown)

In [ ]:
url = 'https://www.ipthailand.go.th/th/trademark-002.html'

In [ ]:
result = await crawler_function(url)

In [ ]:
print(result.html)

In [ ]:
print(result.markdown)

In [ ]:
from IPython.display import Markdown, display

In [ ]:
display(Markdown(result.markdown[:5000])) 

## Crawl into another URL

In [ ]:
soup = BeautifulSoup(result.html, 'html.parser')
print(soup.prettify())

In [ ]:
divs = soup.find_all('div', class_='uk-width-medium-1-2')

In [ ]:
for d in divs:
    print('https://www.ipthailand.go.th/'+d.find('a', class_='zx')['href'])

In [ ]:
divs[0].find('a', class_='zx')['href']

In [ ]:
'https://www.ipthailand.go.th/'+divs[0].find('a', class_='zx')['href']

In [ ]:
async def content_crawler(url):
    config = CrawlerRunConfig(
        scraping_strategy=LXMLWebScrapingStrategy(),
        verbose=False
    )
    async with AsyncWebCrawler() as crawler:
        result = await crawler.arun(url, config=config)

    return result

In [ ]:
result = await content_crawler('https://www.ipthailand.go.th/'+divs[4].find('a', class_='zx')['href'])

In [ ]:
print(result.markdown)
# display(Markdown(result.markdown[:50000])) 

In [ ]:
datas = []
result = await crawler_function('https://www.ipthailand.go.th/th/trademark-002.html')
divs = soup.find_all('div', class_='uk-width-medium-1-2')
for d in divs:
    datas.append(await content_crawler('https://www.ipthailand.go.th/'+d.find('a', class_='zx')['href']))

# Transform into DataFrame

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({
    'markdown' : [row.markdown for row in datas],
    'content' : [row.html for row in datas]
})

In [ ]:
df

In [ ]:
df['markdown'][0]

In [ ]:
display(Markdown(df['markdown'][1][:25000]))

# Scraping PDF Files

In [ ]:
result = await crawler_function('https://tpso.go.th/articles')

In [ ]:
# display(Markdown(result.markdown))

In [ ]:
soup = BeautifulSoup(result.html, 'html.parser')
print(soup.prettify())

In [ ]:
divs = soup.find_all('a', class_='underline-offset-2 hover:underline')

In [ ]:
divs

In [ ]:
divs[0]['href']

In [ ]:
import os

In [ ]:
downloads_path = './pdf_downloads'
os.makedirs(downloads_path, exist_ok=True)

## More Scraping

In [ ]:
datas = []
for d in divs:
    datas.append(await crawler_function('https://tpso.go.th'+d['href']))
    # print('https://tpso.go.th'+d['href'])

In [ ]:
divs

## Download PDF

In [ ]:
for row in datas:
    for uri in row.links['internal']:
        if 'uploads' in uri['href']: 
            name = uri['href'].split('/')[-1]
            res = requests.get(uri['href'])
            pdf = open(downloads_path+'/'+row.url.split('/')[-1]+'.pdf', 'wb')
            pdf.write(res.content)
            pdf.close()

# Transform into DataFrame

In [ ]:
df2 = pd.DataFrame({
    'markdown' : [row.markdown for row in datas],
    'content' : [row.html for row in datas],
    'pdf_file' : [row.url.split('/')[-1]+'.pdf' for row in datas for uri in row.links['internal'] if 'uploads' in uri['href']]
})

In [ ]:
df2

In [ ]:
downloads_path = './json_file'
os.makedirs(downloads_path, exist_ok=True)

# Save to json line

In [ ]:
df.to_json('./json_file/ipthailand.jsonl', orient='records', lines=True)
df2.to_json('./json_file/tpso.jsonl', orient='records', lines=True)